## Compact and full LLM szerver for the Chrome Extension

#### The elte-ik-llm-dlite notebook contains the installation and the experiments for the project.
#### This notebook is the practical runnable server.

### (0) Server starting

d:

cd D:\elte-ik-adatbanyaszat

venv\Scripts\activate

jupyter notebook

### (1) Load the pretrained model

Load and prepare the dlite-v2-1_5b LLM to the GPU

In [1]:
import sys
sys.path.insert(1, './dlite-v2-1_5b/')
from instruct_pipeline import InstructionTextGenerationPipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("dlite-v2-1_5b", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("dlite-v2-1_5b", device_map="cuda", torch_dtype=torch.bfloat16)

generate_text = InstructionTextGenerationPipeline(model=model, tokenizer=tokenizer)

### (2) The REST server

Initialise Flask REST CORS server for recive the process requests

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import subprocess
import re

app = Flask(__name__)
CORS(app)

def req_length(text):
    tokens = tokenizer(text)
    tokens_length = len(tokens.attention_mask)
    print('LOG: Input length: ' + str(tokens_length))
    return tokens_length

def req_to_json(title, text):
    return jsonify({title : text})

def req_process(command, text):
    req = command + ' ' + text
    print('LOG: ' + req)
    return generate_text(req)

def req_clean(text):
    cleaned_text = text.replace('\n', ' ').replace('\r', ' ')
    cleaned_text = re.sub(r'[^\w\s.]', '', cleaned_text)
    return cleaned_text

@app.route('/sum', methods=['POST'])
def sum_req():
    data = request.get_json()
    print(data)
    text = req_clean(data.get('text', ''))
    if(req_length(text) <= 1024):
        return req_to_json('response', req_process('Summarize in one sentence:', text))
        #return req_to_json('response', "TEST TEXT")
    else:
        return req_to_json('error', 'Too long text. Limit is 1024 token.')

@app.route('/key', methods=['POST'])
def key_req():
    data = request.get_json()
    text = req_clean(data.get('text', ''))
    if(req_length(text) <= 1024):
        return req_to_json('response', req_process('Give some keywords from this text:', text))
    else:
        return req_to_json('error', 'Too long text. Limit is 1024 token.')

@app.route('/title', methods=['POST'])
def title_req():
    data = request.get_json()
    text = req_clean(data.get('text', ''))
    if(req_length(text) <= 1024):
        return req_to_json('response', req_process('Give the most important word in the text:', text))
    else:
        return req_to_json('error', 'Too long text. Limit is 1024 token.')

@app.route('/length', methods=['POST'])
def length_req():
    data = request.get_json()
    text = req_clean(data.get('text', ''))
    ret = {'response': req_length(text)}
    return jsonify(ret)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/May/2024 01:33:51] "POST /length HTTP/1.1" 200 -


LOG: Input length: 67
LOG: Input length: 67
LOG: Give the most important word in the text: Jupiter the largest planet in our solar system commands attention with its immense size and swirling clouds of gas. Named after the king of the Roman gods Jupiters colossal presence dominates the space around it. Its massive magnetic field creates a protective shield deflecting harmful solar winds and radiation making it a formidable guardian of the inner solar system.


127.0.0.1 - - [13/May/2024 01:34:16] "POST /title HTTP/1.1" 200 -


LOG: Input length: 67
LOG: Give some keywords from this text: Jupiter the largest planet in our solar system commands attention with its immense size and swirling clouds of gas. Named after the king of the Roman gods Jupiters colossal presence dominates the space around it. Its massive magnetic field creates a protective shield deflecting harmful solar winds and radiation making it a formidable guardian of the inner solar system.


...

127.0.0.1 - - [13/May/2024 02:25:50] "POST /title HTTP/1.1" 200 -


LOG: Input length: 242
LOG: Give some keywords from this text: Cats beloved companions to millions around the world captivate us with their grace independence and mysterious allure. Domesticated for thousands of years these enigmatic creatures have woven themselves into the fabric of human society earning their place as cherished pets and valued members of the family.  Known for their agility and keen hunting instincts cats are skilled predators capable of navigating even the most precarious of environments with ease. Their lithe bodies sharp claws and acute senses make them formidable hunters capable of stalking prey with silent precision.  Yet despite their predatory nature cats also possess a softer side. They are affectionate animals forming strong bonds with their human companions and often seeking out cuddles and attention. Their soothing purrs and gentle nuzzles bring comfort and joy to those fortunate enough to share their homes.  Cats come in a variety of breeds colors and per

127.0.0.1 - - [13/May/2024 02:26:15] "POST /key HTTP/1.1" 200 -


{'text': "Cats, beloved companions to millions around the world, captivate us with their grace, independence, and mysterious allure. Domesticated for thousands of years, these enigmatic creatures have woven themselves into the fabric of human society, earning their place as cherished pets and valued members of the family.  Known for their agility and keen hunting instincts, cats are skilled predators capable of navigating even the most precarious of environments with ease. Their lithe bodies, sharp claws, and acute senses make them formidable hunters, capable of stalking prey with silent precision.  Yet, despite their predatory nature, cats also possess a softer side. They are affectionate animals, forming strong bonds with their human companions and often seeking out cuddles and attention. Their soothing purrs and gentle nuzzles bring comfort and joy to those fortunate enough to share their homes.  Cats come in a variety of breeds, colors, and personalities, each with its own unique q

127.0.0.1 - - [13/May/2024 02:26:43] "POST /sum HTTP/1.1" 200 -
